# PKL Visualization: Depth, Segmentation, and Asset

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from matplotlib import colors

In [ ]:
cwd = Path.cwd()
repo_root = cwd
for p in [cwd] + list(cwd.parents):
    if (p / 'M^3-Verse_pipeline').exists():
        repo_root = p
        break
depth_pkl_path = str(repo_root / 'M^3-Verse/data/scene_2/state_0/depth/frame_86.pkl')
seg_pkl_path = str(repo_root / 'M^3-Verse/data/scene_2/state_0/segmentation/frame_86.pkl')
asset_pkl_path = str(repo_root / '2_object_descriptions/assets/Bed_20/scene_2_state_0_RGB_frame_86.pkl')
rgb_root = str(repo_root / 'M^3-Verse/data')


In [ ]:
def visualize_depth_map(pkl_path):
    with open(pkl_path, 'rb') as f:
        depth_map = pickle.load(f)
    try:
        depth_map = np.array(depth_map)
    except Exception:
        return
    mask = np.isfinite(depth_map)
    if not np.any(mask):
        return
    p_low, p_high = np.percentile(depth_map[mask], [2, 98])
    if p_low == p_high:
        p_low = float(np.min(depth_map[mask]))
        p_high = float(np.max(depth_map[mask]))
    clipped = np.clip(depth_map, p_low, p_high)
    norm = (clipped - p_low) / (p_high - p_low + 1e-12)
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    im0 = axs[0].imshow(norm, cmap='viridis')
    axs[0].set_title('Depth (Viridis, robust)')
    axs[0].axis('off')
    fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
    im1 = axs[1].imshow(norm, cmap='magma', norm=colors.PowerNorm(gamma=0.6))
    axs[1].set_title('Depth (Magma, gamma=0.6)')
    axs[1].axis('off')
    fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_segmentation_map(pkl_path):
    with open(pkl_path, 'rb') as f:
        segmentation_data = pickle.load(f)
    if not isinstance(segmentation_data, dict):
        return
    if 'instance_segmentation' not in segmentation_data:
        return
    img = segmentation_data['instance_segmentation']
    if not isinstance(img, np.ndarray) or img.ndim != 3 or img.shape[2] != 3:
        return
    if img.dtype != np.uint8:
        img = img.astype(np.uint8)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h = hsv[..., 0]
    s = hsv[..., 1]
    v = hsv[..., 2]
    fig, axs = plt.subplots(1, 4, figsize=(16, 4))
    axs[0].imshow(img)
    axs[0].set_title('Segmentation RGB')
    axs[0].axis('off')
    im1 = axs[1].imshow(h, cmap='hsv')
    axs[1].set_title('Hue (HSV)')
    axs[1].axis('off')
    fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
    im2 = axs[2].imshow(s, cmap='magma')
    axs[2].set_title('Saturation')
    axs[2].axis('off')
    fig.colorbar(im2, ax=axs[2], fraction=0.046, pad=0.04)
    im3 = axs[3].imshow(v, cmap='viridis')
    axs[3].set_title('Value')
    axs[3].axis('off')
    fig.colorbar(im3, ax=axs[3], fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()

In [ ]:
def apply_mask(image, mask):
    if mask.dtype != bool:
        mask = mask.astype(bool)
    result = np.zeros_like(image)
    result[mask] = image[mask]
    return result

def visualize_asset(pkl_path, rgb_root):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    rgb_rel = data.get('image_path')
    mask = data.get('mask')
    if rgb_rel is None or mask is None:
        print('Error: asset pkl missing image_path or mask')
        return
    rgb_path = os.path.join(rgb_root, rgb_rel)
    img_bgr = cv2.imread(rgb_path)
    if img_bgr is None:
        print(f'Error: cannot read RGB image: {rgb_path}')
        return
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    masked = apply_mask(img_rgb, mask)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(img_rgb)
    axs[0].axis('off')
    axs[0].set_title('Asset: Original RGB')
    axs[1].imshow(masked)
    axs[1].axis('off')
    axs[1].set_title('Asset: Masked RGB')
    plt.show()

In [ ]:
visualize_depth_map(depth_pkl_path)

In [ ]:
visualize_segmentation_map(seg_pkl_path)

In [ ]:
visualize_asset(asset_pkl_path, rgb_root)